# 🧉 Transcribir podcast con timestamps, identificar multiples speakers, y extraer el texto de uno solo
- **machinelearnear** 📺 https://www.youtube.com/c/machinelearnear

### definir input

In [ ]:
hf_token = "<aca-va-el-hf-token>" # https://huggingface.co/settings/tokens
youtube_video = 'https://www.youtube.com/watch?v=MZEnIUQGuDg&ab_channel=ElPelucaMilei' # El Método Rebord #48 - Alejandro Dolina
youtube_video_candidato = 'https://www.youtube.com/shorts/7iztngtoe-0' # shorts, <= 60 segundos

### instalar las cosas

In [ ]:
!pip install git+https://github.com/m-bain/whisperx.git;
!python3 -m pip install -U yt-dlp;

### download del video desde YT

In [ ]:
!python -m yt_dlp --output "audio.%(ext)s" --extract-audio --audio-format wav $youtube_video

### transcribir y hacer speaker diarization

In [ ]:
!whisperx audio.wav --hf_token $hf_token --model large-v2 --language es --align_model WAV2VEC2_ASR_LARGE_LV60K_960H --diarize --min_speakers 2

### mergear segmentos basandonos en los speakers

In [ ]:
import json

# Load the JSON file
with open("audio.json", "r") as file:
    data = json.load(file)

In [ ]:
data['segments'][0]['speaker']

### hacer speaker verification tomando un clip de audio del candidato como baseline

In [ ]:
!python -m yt_dlp --output "audio_candidato.%(ext)s" --extract-audio --audio-format wav $youtube_video_candidato

In [ ]:
import torch
from pyannote.audio.pipelines.speaker_verification import PretrainedSpeakerEmbedding
model = PretrainedSpeakerEmbedding(
    "speechbrain/spkrec-ecapa-voxceleb",
    device=torch.device("cuda"))

In [ ]:
from pyannote.audio import Audio
from pyannote.core import Segment
from scipy.spatial.distance import cdist

audio = Audio(sample_rate=16000, mono="downmix")

In [ ]:
# extract embedding for a speaker speaking between t=3s and t=6s
candidato = Segment(1., 15.)
waveform_candidato, sample_rate = audio.crop("audio_candidato.wav", candidato)
embedding_candidato = model(waveform_candidato[None])

In [ ]:
import time

stime = time.time()

In [ ]:
import time

stime = time.time()

for segment in data['segments']:
  # extract embedding for a speaker speaking between t=Xs and t=Ys
  target_speaker = Segment(segment['start'], segment['end'])
  waveform_target, sample_rate = audio.crop("audio.wav", target_speaker)
  embedding_target = model(waveform_target[None])

  # compare embeddings using "cosine" distance
  distance = cdist(embedding_candidato, embedding_target, metric="cosine")

  # save back the info to the dict
  segment['is_candidate'] = True if distance[0][0] <= 0.5 else False

print(f'total time: {time.time()-stime:.2f} seconds')

In [ ]:
for segment in data['segments']:
  if segment['is_candidate']:
    print(f"Start: {segment['start']}, End: {segment['end']}, Text: {segment['text'].strip()}")